# 05 - Cálculo de Inferencia (Predicciones)

## Descripción General

Este notebook realiza el proceso de **inferencia** utilizando un modelo de Machine Learning previamente entrenado. El objetivo es generar predicciones sobre datos de prueba (test) que no fueron utilizados durante el entrenamiento del modelo.

### Flujo del Notebook

1. **Importaciones**: Carga de librerías y módulos del proyecto
2. **Carga del Pipeline**: Lectura del modelo entrenado desde disco
3. **Preparación de Datos**: Carga y procesamiento de datos para inferencia
4. **Generación de Predicciones**: Aplicación del modelo sobre los datos de test
5. **Guardado de Resultados**: Exportación de las predicciones a archivo

### Requisitos Previos

- Haber ejecutado el entrenamiento del modelo (notebook `04_model_tuning_training.ipynb`)
- Tener el archivo del pipeline guardado en la ubicación especificada en `config.py`
- Datos raw disponibles en `data/raw/train.csv`

---

## 1. Importación de Librerías y Módulos

Se importan las librerías necesarias:
- **pandas/numpy**: Manipulación de datos
- **product_development.config**: Configuración global del proyecto (features, rutas, parámetros)
- **product_development.dataset**: Funciones para carga y preparación de datos
- **product_development.modeling.train**: Funciones de entrenamiento y carga de pipelines
- **product_development.modeling.predict**: Funciones de predicción e inferencia

In [8]:
# Importaciones
import sys
sys.path.append('..')

import pandas as pd
import numpy as np

# Importar módulos del paquete product_development
from product_development.config import (
    FEATURES, TARGET, CATEGORICAL_VARS, RAW_DATA_DIR, 
    PIPELINE_FILE, PREDICTIONS_FILE, TRAIN_TEST_SPLIT_RATIO
)
from product_development.dataset import load_raw_data, prepare_dataset
from product_development.modeling.train import load_pipeline
from product_development.modeling.predict import (
    predict, prepare_inference_data, save_predictions, run_inference
)

## 2. Carga del Pipeline Entrenado

Se carga el pipeline de ML previamente entrenado desde el archivo especificado en `PIPELINE_FILE`.

El pipeline incluye:
- Preprocesamiento de datos (transformaciones, encoding de variables categóricas)
- Modelo de predicción entrenado

> **Nota**: El archivo del pipeline debe existir. Si no existe, ejecutar primero el notebook de entrenamiento.

In [9]:
# 1. Cargar pipeline entrenado
sales_pipeline = load_pipeline(PIPELINE_FILE)
print("Pipeline cargado correctamente.")

2025-11-29 12:35:39.137 | INFO     | product_development.modeling.train:load_pipeline:648 - Cargando pipeline desde C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\models\sales_pipeline.pkl
Pipeline cargado correctamente.


## 3. Carga y Preparación de Datos para Inferencia

### Proceso:
1. **Carga de datos raw**: Se lee el archivo CSV original
2. **Preparación del dataset**: Se aplican transformaciones iniciales (limpieza, feature engineering básico)
3. **Split de datos**: Se obtiene el 20% final de los datos para test (consistente con el split de entrenamiento)

### Parámetros:
- `TRAIN_TEST_SPLIT_RATIO`: Proporción de datos para entrenamiento (80%)
- Los datos de test son el 20% restante que NO fueron usados en entrenamiento

In [10]:
# 2. Cargar y preparar datos para inferencia
raw_data = load_raw_data(RAW_DATA_DIR / "train.csv")
dataset = prepare_dataset(raw_data)

# Obtener el 20% final para test (mismo split que entrenamiento)
n_samples = len(dataset)
n_train = int(n_samples * TRAIN_TEST_SPLIT_RATIO)

test_df = dataset.iloc[n_train:].copy()

print(f"Total registros: {n_samples}")
print(f"Registros de test: {len(test_df)}")
test_df.head()

2025-11-29 12:35:39.149 | INFO     | product_development.dataset:load_raw_data:49 - Cargando datos crudos desde C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\raw\train.csv
2025-11-29 12:35:39.498 | INFO     | product_development.dataset:load_raw_data:51 - Cargados 913000 registros
2025-11-29 12:35:39.498 | INFO     | product_development.dataset:add_temporal_features:69 - Agregando características temporales
2025-11-29 12:35:39.635 | INFO     | product_development.dataset:convert_categorical_types:92 - Convirtiendo columnas categóricas a tipo object
Total registros: 913000
Registros de test: 182600


,date,store,item,sales,year,month,day_of_week_name
730400,2013-01-01,1,41,6,2013,1,Tuesday
730401,2013-01-02,1,41,15,2013,1,Wednesday
730402,2013-01-03,1,41,5,2013,1,Thursday
730403,2013-01-04,1,41,9,2013,1,Friday
730404,2013-01-05,1,41,13,2013,1,Saturday


## 4. Preparación de Features para Inferencia

La función `prepare_inference_data()` extrae únicamente las columnas de features necesarias para el modelo, excluyendo la variable objetivo y otras columnas no requeridas.

**Features utilizadas**: Definidas en `FEATURES` dentro de `config.py`

In [11]:
# 3. Preparar datos para inferencia
X_test = prepare_inference_data(test_df)
print(f"Features preparadas: {X_test.shape}")
X_test.head()

2025-11-29 12:35:39.728 | INFO     | product_development.modeling.predict:prepare_inference_data:67 - Preparando datos para inferencia
Features preparadas: (182600, 5)


,store,item,year,month,day_of_week_name
730400,1,41,2013,1,Tuesday
730401,1,41,2013,1,Wednesday
730402,1,41,2013,1,Thursday
730403,1,41,2013,1,Friday
730404,1,41,2013,1,Saturday


## 5. Generación de Predicciones

Se utiliza la función `predict()` que:
1. Aplica las transformaciones del pipeline a los datos de entrada
2. Genera las predicciones usando el modelo entrenado
3. Retorna un array con las predicciones

> **Nota**: El pipeline maneja internamente el preprocesamiento, por lo que los datos de entrada deben estar en formato "raw" (sin transformar).

In [12]:
# 4. Generar predicciones
y_test_pred = predict(sales_pipeline, X_test)

print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Primeras 10 predicciones: {y_test_pred[:10]}")

2025-11-29 12:35:39.774 | INFO     | product_development.modeling.predict:predict:47 - Generando predicciones para 182600 muestras


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


2025-11-29 12:35:39.990 | INFO     | product_development.modeling.predict:predict:49 - Predicciones generadas exitosamente
Predicciones generadas: 182600
Primeras 10 predicciones: [27.269253 27.235146 29.151873 31.31193  32.956093 35.11257  23.533398
 27.269253 27.235146 29.151873]


## 6. Guardado de Predicciones

Las predicciones se guardan en un archivo CSV que incluye:
- Los datos originales del conjunto de test
- Una columna adicional con las predicciones generadas

**Ubicación del archivo**: Definida en `PREDICTIONS_FILE` (típicamente `data/processed/test_predictions.csv`)

In [13]:
# 5. Guardar predicciones
output_df = save_predictions(test_df, y_test_pred, PREDICTIONS_FILE)

print(f"Archivo guardado en: {PREDICTIONS_FILE}")
output_df.head()

2025-11-29 12:35:39.999 | INFO     | product_development.modeling.predict:save_predictions:115 - Guardando predicciones en C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\processed\test_predictions.csv
2025-11-29 12:35:42.496 | INFO     | product_development.modeling.predict:save_predictions:123 - Predicciones guardadas: 182600 registros
Archivo guardado en: C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\processed\test_predictions.csv


,date,store,item,sales,year,month,day_of_week_name,sales_pred
730400,2013-01-01,1,41,6,2013,1,Tuesday,27.269253
730401,2013-01-02,1,41,15,2013,1,Wednesday,27.235146
730402,2013-01-03,1,41,5,2013,1,Thursday,29.151873
730403,2013-01-04,1,41,9,2013,1,Friday,31.311930
730404,2013-01-05,1,41,13,2013,1,Saturday,32.956093


---

## Resumen

Este notebook completa el ciclo de ML realizando inferencia sobre datos de prueba:

| Paso | Descripción | Output |
|------|-------------|--------|
| 1 | Importaciones | Módulos cargados |
| 2 | Carga Pipeline | Modelo listo para predicción |
| 3 | Preparación Datos | DataFrame de test |
| 4 | Preparación Features | X_test con features del modelo |
| 5 | Predicciones | Array de predicciones |
| 6 | Guardado | Archivo CSV con resultados |

### Siguiente Paso
Las predicciones pueden ser evaluadas comparándolas con los valores reales para calcular métricas de rendimiento del modelo.